In [ ]:
!nvidia-smi

Fri Jan 12 09:50:58 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   77C    P0              35W /  70W |    401MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
Fri Jan 12 09:44:14 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   77C    P0              35W /  7

In [ ]:
import cv2
import os
import random
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import torch
import tqdm
from typing import Optional
from PIL import Image
from tqdm import tqdm_notebook
import torch.nn as nn
import torchvision.models as models


from matplotlib import pyplot as plt
from torch import nn
from torch import optim
from torch.nn import functional as F
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets, transforms
from torchvision import transforms as tfs
from torchvision.transforms import v2
from torchvision.datasets import ImageFolder
from tqdm.notebook import tqdm




In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
#WITH AUGMENTATION2
training_folder='/content/drive/MyDrive/archive/data/training'
validation_folder="/content/drive/MyDrive/archive/data/validation"

# Create a list of transformations
transformations_list = [
    transforms.RandomHorizontalFlip(),
    v2.RandomHorizontalFlip(),
    v2.RandomRotation(30),
    v2.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    v2.RandomResizedCrop(224),
    v2.ToTensor(),
    v2.Resize(600),
    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
]

# Compose the transformations
transform_gen = v2.Compose(transformations_list)

transform_orig = transforms.Compose([
transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])


train_dataset = datasets.ImageFolder(root=training_folder, transform=transform_orig)
train_dataset_aug = datasets.ImageFolder(root=training_folder, transform=transform_gen)
train_data = torch.utils.data.ConcatDataset([train_dataset, train_dataset_aug])



val_dataset = datasets.ImageFolder(root=validation_folder, transform=transform_orig)
val_dataset_aug = datasets.ImageFolder(root=validation_folder, transform=transform_gen)
validation_data = torch.utils.data.ConcatDataset([val_dataset, val_dataset_aug])


train_loader = DataLoader(train_data, batch_size=4, shuffle=True, num_workers=2)
val_loader = DataLoader(validation_data, batch_size=4, shuffle=False, num_workers=2)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
#x_batch.shape, y_batch.shape #3-канальные изображения 600х600

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/v2/_deprecated.py:43: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.
  warnings.warn(


In [ ]:
#NO AUGMENTATION
training_folder='/content/drive/MyDrive/archive/data/training'
validation_folder="/content/drive/MyDrive/archive/data/validation"
transform_orig = transforms.Compose([
transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])


train_dataset = datasets.ImageFolder(root=training_folder, transform=transform_orig)


val_dataset = datasets.ImageFolder(root=validation_folder, transform=transform_orig)


train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False, num_workers=2)

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
from torch.nn.functional import cross_entropy
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class MyModule(nn.Module):
  def __init__(self):
    super(MyModule, self).__init__()
    self.conv_skip_con = nn.Conv2d(3, 128, kernel_size = 105, stride = 105, padding = 1)
    self.conv1 = nn.Conv2d(3, 32, kernel_size = 5, stride = 3, padding = 1)
    self.bn1 = nn.BatchNorm2d(32)
    self.pool = nn.MaxPool2d(kernel_size = 2, stride = 2)

    self.conv2 = nn.Conv2d(32, 64, kernel_size = 5, stride = 3, padding = 1)
    self.bn2 = nn.BatchNorm2d(64)

    self.conv3 = nn.Conv2d(64, 128, kernel_size = 5, stride = 3, padding = 1)
    self.bn3 = nn.BatchNorm2d(128)
    self.fc1 = nn.Linear(128*2*2, 500)
    self.dropout = nn.Dropout(0.7)
    self.fc2 = nn.Linear(500, 2)

  def forward(self, x):
    id= x

    id2 = self.pool(F.relu(self.conv_skip_con(id)))
    out = self.pool(F.relu(self.bn1(self.conv1(x))))
    out = self.pool(F.relu(self.bn2(self.conv2(out))))

    out = self.pool(F.relu(self.bn3(self.conv3(out))))
    out += id2
    out = out.view(-1, 128*2*2)
    out=self.dropout(F.relu(self.fc1(out)))
    out = F.softmax(self.fc2(out), dim=1)
    return out

lr=1e-2
history = []
net = MyModule().to(device)
net.requires_grad_()
batch_size=32

loss_fn=nn.CrossEntropyLoss()
criterion = nn.CrossEntropyLoss()
epochs = 10
lr=1e-2
history = []
net=net.to(device)
net.requires_grad_()
batch_size=32

loss_fn=nn.CrossEntropyLoss()
criterion = nn.CrossEntropyLoss()

In [ ]:
optimizer=torch.optim.Adam(net.parameters(), lr=0.00001)

def trainer(train_loader, val_loader, net):
  losses_train = []
  losses_val=[]
  accuracy_val=[]
  accuracy_train=[]
  for epoch in range(epochs):
      net.train()
      running_loss = 0
      correct = 0
      num = 0
      for i, (x_batch, y_batch) in enumerate(tqdm_notebook(train_loader)):
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)

        logits = net(x_batch)

        loss=criterion(logits, y_batch)
        history.append(loss.item())
        running_loss+=loss.item()

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        _, preds = torch.max(logits, axis=-1)
        correct += (preds == y_batch).sum().item()
        num += len(x_batch)

      losses_train.append(running_loss/len(train_loader))
      accuracy_train.append(correct / num)
      print("losses train:", losses_train)
      print("accuracy train: ", accuracy_train)
      net.eval()
      running_loss=0
      correct=0
      num=0
      with torch.no_grad():
        for xb, yb in tqdm(val_loader):
          xb, yb = xb.to(device), yb.to(device)
          probs = net(xb)
          running_loss += criterion(probs, yb).item()

          _, preds = torch.max(probs, axis=-1)
          correct += (preds == yb).sum().item()
          num += len(xb)

      losses_val.append(running_loss / len(val_loader))
      accuracy_val.append(correct / num)
      print("losses validation:", losses_val)
      print('accuracy validation:', accuracy_val)
trainer(train_loader, val_loader, net)


<ipython-input-139-bb7779d59fc3>:13: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i, (x_batch, y_batch) in enumerate(tqdm_notebook(train_loader)):


  0%|          | 0/461 [00:00<?, ?it/s]

losses train: [0.7047979416661045]
accuracy train:  [0.5285171102661597]


  0%|          | 0/50 [00:00<?, ?it/s]

losses validation: [0.6885883212089539]
accuracy validation: [0.535]


...

  0%|          | 0/461 [00:00<?, ?it/s]

losses train: [0.7047979416661045, 0.7012468785241472, 0.6891914646175575, 0.695592014753379, 0.676827751633402, 0.6632874710828777, 0.665906182015537, 0.6557055614775535, 0.6502831405000418]
accuracy train:  [0.5285171102661597, 0.5437262357414449, 0.5719717544812601, 0.5632808256382401, 0.5871808799565453, 0.6056491037479631, 0.6051059206952744, 0.6186854970124932, 0.6262900597501357]


  0%|          | 0/50 [00:00<?, ?it/s]

losses validation: [0.6885883212089539, 0.6932170552015304, 0.6855622237920761, 0.6824252021312713, 0.6831310606002807, 0.6934885519742966, 0.6908855938911438, 0.6832342714071273, 0.6757786601781846]
accuracy validation: [0.535, 0.545, 0.585, 0.58, 0.605, 0.54, 0.57, 0.57, 0.6]


  0%|          | 0/461 [00:00<?, ?it/s]

In [ ]:
#визуализация
def plot_trainig(losses_train, losses_val, accuracy_val, accuracy_train):
    plt.figure(figsize=(12, 9))
    plt.subplot(2, 1, 1)
    plt.xlabel("epoch")
    plt.plot(losses_train, label="train_loss")
    plt.plot(losses_val, label="valid_loss")
    plt.legend()

    plt.subplot(2, 1, 2)
    plt.xlabel("epoch")
    plt.plot(accuracy_val, label="valid accuracy")
    plt.plot(accuracy_train, label="training_accuracy")
    plt.legend()
print(plot_trainig(losses_train, losses_val, accuracy_val, accuracy_train))

NameError: name 'losses_train' is not defined